# Example: Breadth-First Search (BFS) on some Simple Trees and Graphs
This example will familiarize students with [Breadth-First Search](https://en.wikipedia.org/wiki/Breadth-first_search) graph traversal with some simple and more complex graphs.

* For some interesting graphs (although they are a bit old now), check out the [Stanford Network Analysis Project](https://snap.stanford.edu/). This group maintains the [Stanford Large Network Dataset Collection](https://snap.stanford.edu/data/index.html), which has many large graphs on which to test various algorithms.

## Setup
This example may use external third-party packages. In the `Include.jl` file, we load our codes to access them in the notebook, set some required paths for this example, and load any required external packages.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a/Manifest.toml`


## Task 1: Build an instance of `MySimpleDirectedGraphModel` for an example graph using an edge list
A graph $\mathcal{G}=\left(\mathcal{V},\mathcal{E}\right)$ can be constructed from an [Adjacency Matrix](https://en.wikipedia.org/wiki/Adjacency_matrix) $\mathbf{A}$, which is a $\dim\mathcal{V}\times\dim\mathcal{V}$ square matrix. However, this is only suitable for small graphs because $\mathbf{A}$ has a high memory overhead (if stored as `64-bit` values). 
* For example, consider a graph with $\dim\mathcal{V}$ = 100000 would require `80 GB` of memory to store in the worst case, which is more than most common machines

In [2]:
𝒱 = 100000;
memory_reqd = (𝒱^2)*8*(1/(1e9)) # units GB

80.0

Instead, only the edges are stored in an [Edge list representation](https://en.wikipedia.org/wiki/Edge_list), which is (typically) a comma-separated value (CSV) file in which each record holds an edge in the graph, and the fields contain `source, target, weight` data for the edge.

* We've built the `readedgesfile(...)` function in `src/Files.jl` and the `MyGraphEdgeModel` type (in `src/Types.jl`) to hold this information. Let's load up an edge list. First, set the path to the edge list file:

In [3]:
# path_to_edge_file = joinpath(_PATH_TO_DATA, "soc-sign-bitcoinalpha.csv");
path_to_edge_file = joinpath(_PATH_TO_DATA, "SimpleGraph.txt");

Next, let's load build a dictionary of `edges,` where the data for the edges (source id, target id, and weight) is stored in a `MyGraphEdgeModel` model. 
* We utilize the `readedgesfile` function from `src/Files.jl` to read the edge data. This function requires the path to the edges file and information about the delimiter and comment characters. It returns a dictionary that holds instances of `MyGraphEdgeModel`

In [12]:
myedges = readedgesfile(path_to_edge_file, delim=',', comment='#')

Dict{Int64, MyGraphEdgeModel} with 7 entries:
  0 => MyGraphEdgeModel(0, 1, 2, 10.0)
  4 => MyGraphEdgeModel(4, 3, 5, 6.0)
  5 => MyGraphEdgeModel(5, 4, 6, 1.0)
  6 => MyGraphEdgeModel(6, 5, 4, 1.0)
  2 => MyGraphEdgeModel(2, 2, 3, 2.0)
  3 => MyGraphEdgeModel(3, 2, 4, 100.0)
  1 => MyGraphEdgeModel(1, 1, 3, 100.0)

Finally, now that we have the `myedges` dictionary, we can build a graph instance. Since this is a directed graph, we'll construct a `MySimpleDirectedGraphModel` instance using it's `build(...)` method in the `src/Factory.jl` file.

In [14]:
dag = build(MySimpleDirectedGraphModel, myedges)

MySimpleDirectedGraphModel(Dict{Int64, MyGraphNodeModel}(5 => MyGraphNodeModel(5), 4 => MyGraphNodeModel(4), 6 => MyGraphNodeModel(6), 2 => MyGraphNodeModel(2), 3 => MyGraphNodeModel(3), 1 => MyGraphNodeModel(1)), Dict((2, 4) => 100, (1, 2) => 10, (1, 3) => 100, (4, 6) => 1, (3, 5) => 6, (5, 4) => 1, (2, 3) => 2), Dict{Int64, Set{Int64}}(5 => Set([4]), 4 => Set([6]), 6 => Set(), 2 => Set([4, 3]), 3 => Set([5]), 1 => Set([2, 3])))

In [20]:
dag.children

Dict{Int64, Set{Int64}} with 6 entries:
  5 => Set([4])
  4 => Set([6])
  6 => Set()
  2 => Set([4, 3])
  3 => Set([5])
  1 => Set([2, 3])

## Task 2: Use our BFS implementation to visit the nodes of the `dag`
The completed implementation of the `BFS` algorithm can be found in the `src/Traversal.jl` file ([check it out!](src/Traversal.jl)). In particular, we'll start at the first node and visit all other nodes in the graph.

In [22]:
start_node = dag.nodes[5] # note the SNAP data is 1-based, we renumber our graph to 1-based

MyGraphNodeModel(5)

To use the `BFS` implementation, pass the graph instance, in this case, `dag,` and the start node to the `BFS(...)` function. The `BFS` algorithm visits all other nodes by visiting all the children of `start_node` and then all the grandchildren of `start_node,` etc.
* The `verbose::Bool` argument tells the `BFS` function to generate a picture of its current progress and write it to the `frames` directory.

In [23]:
#visited = Set{Int64}();
#DFS(dag, start_node, visited; verbose = true);
BFS(dag, start_node; verbose = true)